# ipycanvas: Python canvas for Jupyter

## https://github.com/martinRenou/ipycanvas

Documentation: https://ipycanvas.readthedocs.io

ipycanvas allows you to draw simple primitives directly from Python like text, lines, polygons, arcs, images etc. This simple toolset allows you to draw literally anything!

- BSD-3-Clause License

**Installation:**

```bash
conda install -c conda-forge ipycanvas
```

### Polygons and shapes

In [ ]:
from time import sleep
from math import pi

import numpy as np

from ipycanvas import Canvas, MultiCanvas, RoughCanvas, hold_canvas

In [ ]:
canvas = Canvas(width=200, height=200)

canvas.fill_style = '#63934e'
canvas.stroke_style = '#4e6393'
canvas.line_width = 5
canvas.fill_polygon([(20, 20), (180, 20), (100, 150)])
canvas.stroke_polygon([(20, 20), (180, 20), (100, 150)])

canvas

In [ ]:
canvas = Canvas(width=150, height=150)

canvas.fill_style = 'white'
canvas.fill_rect(0, 0, 150, 150)
canvas.fill_style = 'purple'

# Cubic curves example
canvas.begin_path()
canvas.move_to(75, 40)
canvas.bezier_curve_to(75, 37, 70, 25, 50, 25)
canvas.bezier_curve_to(20, 25, 20, 62.5, 20, 62.5)
canvas.bezier_curve_to(20, 80, 40, 102, 75, 120)
canvas.bezier_curve_to(110, 102, 130, 80, 130, 62.5)
canvas.bezier_curve_to(130, 62.5, 130, 25, 100, 25)
canvas.bezier_curve_to(85, 25, 75, 37, 75, 40)
canvas.fill()

canvas

### Lines

In [ ]:
canvas = Canvas(width=200, height=200)

canvas.fill_style = 'white'
canvas.fill_rect(0, 0, 200, 200)

n = 100
x = np.linspace(0, 200, n)
y = np.random.randint(200, size=n)

points = np.stack((x, y), axis=1)

canvas.stroke_lines(points)

canvas

### Fractal tree: Generate a new fractal tree on button click

In [ ]:
from math import pi
from random import uniform

from ipywidgets import Button

canvas = Canvas(width=800, height=600)

def recursive_draw_leaf(canvas, length, r_angle, r_factor, l_angle, l_factor):
    canvas.stroke_line(0, 0, 0, -length)
    canvas.translate(0, -length)

    if length > 5 :
        canvas.save()

        canvas.rotate(r_angle)
        recursive_draw_leaf(canvas, length * r_factor, r_angle, r_factor, l_angle, l_factor)

        canvas.restore()

        canvas.save()

        canvas.rotate(l_angle)
        recursive_draw_leaf(canvas, length * l_factor, r_angle, r_factor, l_angle, l_factor)

        canvas.restore()


def draw_tree(canvas):
    with hold_canvas(canvas):
        canvas.save()

        canvas.clear()
        
        canvas.fill_style = 'white'
        canvas.fill_rect(0, 0, 800, 600)

        canvas.translate(canvas.width / 2., canvas.height)

        canvas.stroke_style = 'black'

        r_factor = uniform(0.6, 0.8)
        l_factor = uniform(0.6, 0.8)

        r_angle = uniform(pi / 10., pi / 5.)
        l_angle = uniform(-pi / 5., -pi / 10.)

        recursive_draw_leaf(canvas, 150, r_angle, r_factor, l_angle, l_factor)

        canvas.restore()

button = Button(description='Generate tree!')

def click_callback(*args, **kwargs):
    global canvas

    draw_tree(canvas)

button.on_click(click_callback)

draw_tree(canvas)

display(canvas)
display(button)

### Custom scatter plot

In [ ]:
import branca


class ScatterPlot(MultiCanvas):
    def __init__(self, x, y, size, color=None, scheme=branca.colormap.linear.RdBu_11, stroke_color='black'):
        super(ScatterPlot, self).__init__(3, width=800, height=600, sync_image_data=True)

        self.color = color
        self.scheme = scheme
        
        self.background_color = '#f7f7f7'

        self.init_plot(x, y)
        
        self.sizes = size
        self.stroke_color = stroke_color

        self.n_marks = min(x.shape[0], y.shape[0], size.shape[0], color.shape[0])

        self.draw()

    def init_plot(self, x, y, color=None, scheme=None):
        self.x = x
        self.y = y
        self.color = color if color is not None else self.color
        self.scheme = scheme if scheme is not None else self.scheme

        padding = 0.1
        padding_x = padding * self.width
        padding_y = padding * self.height

        self.drawarea = (drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y) = (padding_x, padding_y, self.width - 2 * padding_x, self.height - 2 * padding_y)

        min_x, min_y, max_x, max_y = np.min(x), np.min(y), np.max(x), np.max(y)

        dx = max_x - min_x
        dy = max_y - min_y

        # Turns a data coordinate into pixel coordinate
        self.scale_x = lambda x: drawarea_max_x * (x - min_x) / dx + drawarea_min_x
        self.scale_y = lambda y: drawarea_max_y * (1 - (y - min_y) / dy) + drawarea_min_y

        # Turns a pixel coordinate into data coordinate
        self.unscale_x = lambda sx: (sx - drawarea_min_x) * dx / drawarea_max_x + min_x
        self.unscale_y = lambda sy: (1 - ((sy - drawarea_min_y) / drawarea_max_y)) * dy + min_y

        self.colormap = None
        if self.color is not None:
            self.colormap = self.scheme.scale(np.min(self.color), np.max(self.color))

    def draw_background(self):
        drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y = self.drawarea

        background = self[0]

        # Draw background
        background.fill_style = self.background_color
        background.global_alpha = 0.3
        background.fill_rect(drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y)
        background.global_alpha = 1

        # Draw grid and ticks
        n_lines = 10
        background.fill_style = 'gray'
        background.stroke_style = '#8c8c8c'
        background.line_width = 1

        for i in range(n_lines):
            j = i / (n_lines - 1)
            line_x = drawarea_max_x * j + drawarea_min_x
            line_y = drawarea_max_y * j + drawarea_min_y

            # Line on the y axis
            background.stroke_line(line_x, drawarea_min_y, line_x, drawarea_max_y + drawarea_min_y)

            # Line on the x axis
            background.stroke_line(drawarea_min_x, line_y, drawarea_max_x + drawarea_min_x, line_y)

            # Draw y tick
            background.text_align = 'right'
            background.text_baseline = 'middle'
            background.fill_text('{0:.2e}'.format(self.unscale_y(line_y)), drawarea_min_x * 0.95, line_y)

            # Draw x tick
            background.text_align = 'center'
            background.text_baseline = 'top'
            background.fill_text('{0:.2e}'.format(self.unscale_x(line_x)), line_x, drawarea_max_y + drawarea_min_y + drawarea_min_y * 0.05)

    def draw(self):
        with hold_canvas(self):
            self.clear()
            plot_layer = self[1]

            plot_layer.save()

            self.draw_background()

            # Draw scatter
            plot_layer.stroke_style = self.stroke_color

            for idx in range(self.n_marks):
                plot_layer.fill_style = self.colormap(self.color[idx])

                mark_x = self.scale_x(self.x[idx])
                mark_y = self.scale_y(self.y[idx])
                mark_size = self.sizes[idx]

                plot_layer.fill_circle(mark_x, mark_y, mark_size)
                plot_layer.stroke_circle(mark_x, mark_y, mark_size)

            plot_layer.restore()


n_points = 1_000

x = np.random.rand(n_points)
y = np.random.rand(n_points)
sizes = np.random.randint(2, 8, n_points)
colors = np.random.rand(n_points) * 10 - 2

plot = ScatterPlot(x, y, sizes, colors, branca.colormap.linear.viridis, stroke_color='white')
plot

### RoughCanvas: Give a "hand-drawn" style to your drawings

In [ ]:
canvas = RoughCanvas()

canvas.fill_style = 'red'
canvas.stroke_style = 'blue'

canvas.stroke_rect(100, 100, 100, 100)
canvas.fill_rect(50, 50, 100, 100)

canvas.stroke_circle(300, 300, 100)
canvas.fill_circle(350, 350, 100)

canvas.stroke_line(200, 200, 300, 300)

canvas

### Game Of Life

In [ ]:
def life_step(x):
    """Game of life step"""
    nbrs_count = sum(np.roll(np.roll(x, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0))
    return (nbrs_count == 3) | (x & (nbrs_count == 2))


def draw(x, canvas, color='black'):
    with hold_canvas(canvas):
        canvas.clear()
        canvas.fill_style = '#FFF0C9'
        canvas.stroke_style = 'white'
        canvas.rough_fill_style = 'solid'
        canvas.fill_rect(-10, -10, canvas.width + 10, canvas.height + 10)
        canvas.rough_fill_style = 'cross-hatch'

        canvas.fill_style = color
        canvas.stroke_style = color

        living_cells = np.where(x)
        
        rects_x = living_cells[0] * n_pixels
        rects_y = living_cells[1] * n_pixels

        canvas.fill_rects(rects_x, rects_y, n_pixels)
        canvas.stroke_rects(rects_x, rects_y, n_pixels)



glider_gun =\
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
 [1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
 [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

x = np.zeros((50, 70), dtype=bool)
x[1:10,1:37] = glider_gun

n_pixels = 15

canvas = RoughCanvas(width=x.shape[0]*n_pixels, height=x.shape[1]*n_pixels)
canvas.fill_style = '#FFF0C9'
canvas.stroke_style = 'white'
canvas.fill_rect(0, 0, canvas.width, canvas.height)

draw(x, canvas, '#5770B3')

display(canvas)

for _ in range(100):
    x = life_step(x)
    draw(x, canvas, '#5770B3')

    sleep(0.1)

### Playing with Perlin Noise

In [ ]:
pattern_source = Canvas(width=50, height=50)

pattern_source.fill_style = '#338ac4'
pattern_source.fill_rect(0, 0, 50, 50)

pattern_source.fill_style = '#3341c4'
pattern_source.fill_circle(50, 50, 5)
pattern_source.fill_circle(0, 0, 5)
pattern_source.fill_circle(50, 0, 5)
pattern_source.fill_circle(0, 50, 5)

pattern_source.fill_style = '#33c4b5'
pattern_source.fill_circle(25, 25, 10)


# Quick and dirty perlin noise function
def dgg(ix, iy, x, y):
    random = 2920 * np.sin(ix * 21942 + iy * 171324 + 8912) * np.cos(ix * 23157 * iy * 217832 + 9758)
    return ((x - ix)*np.cos(random) + (y - iy)*np.sin(random));


def perlin(x,y):
    x0, y0 = np.array(x).astype(int),np.array(y).astype(int)
    n0, n1 = dgg(x0, y0, x, y), dgg((x0 + 1), y0, x, y)
    ix0 = (n1 - n0) * (x - x0) + n0
    n0, n1 = dgg(x0, (y0 + 1), x, y), dgg((x0 + 1), (y0 + 1), x, y)
    return (((n1 - n0) * (x - x0) + n0) - ix0) * (y-y0) + ix0


canvas = Canvas(width=500, height=500)
canvas.global_alpha = 0.1
pattern = canvas.create_pattern(pattern_source)
canvas.fill_style = pattern
display(canvas)

# Some particles
n_particles = 4100
x = np.array(np.random.rayleigh(250, n_particles))
y = np.array(np.random.rayleigh(250, n_particles))
size = np.random.randint(1, 3, n_particles)


def draw_particles():
    canvas.fill_rects(x, y, size)


def update_particle_locations():
    global x, y
    angles = perlin(x/35, y/35) * 3
    x += np.sin(angles) * 0.3
    y += np.cos(angles) * 0.3


for _ in range(200):
    update_particle_locations()
    draw_particles()

    sleep(0.01)